# Testing False Positives of Benign Flight
This notebook looks at false positive rates in the benign case by using a set
of 5 flights where 4 determine the error bounds and 1 is used for testing.

The data files can be found in the data subdirectory. Only interim data will be
available on github to preserve storage. For raw log files please contact me at
srimoungchanh.bailey@ku.edu.

## Loading Data and Parsing Error Margins

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import pandas as pd
import numpy as np
from confirmation.process import process_SNS
from os.path import exists
from confirmation.process import geodetic2ned, change_in_signal
from confirmation.process import length, body_to_earth2D, low_pass_filter
from confirmation.process import signal_match_and_cumsum

#Data used for comparisons later
max_errors = []
errors_north = []
errors_east = []

#Convert log file from Mission Planner to usable CSV
data_dir = "../data/"
files = ["2023-01-26-Delivery (" + str(x) + ").log" for x in range(1, 6)] #list of files to process

for file in files:
    output = data_dir + "interim/" + file[:-3] + "csv"

    #Check if file exists already and if it does skip processing and read the file
    if not exists(output):
        #Open log file and parse out the SNS data into pandas dataframes
        df = process_SNS(data_dir + "raw/" + file, output)
    else:
        df = pd.read_csv(output)

    #All of the data is numeric and should be treated as such
    df = df.apply(pd.to_numeric)

    #Set index to be the timestamps so we can drop duplicate rows
    #then reset index to be row number
    df.set_index("TimeUS",inplace=True)
    df = df.drop_duplicates()
    df.reset_index(inplace=True)

    #Get rid of duplicate rows and convert timestamps to datetime
    ofMS = df[df.ofMS.shift() != df.ofMS][["ofMS","frF","frR","brF","brR", "rf", "m00", "m10"]].reset_index()
    ofMS['ofMS'] = pd.to_datetime(ofMS['ofMS'], unit='ms')
    gpsMS = df[df.gpsMS.shift() != df.gpsMS][["gpsMS","lat","lng","gpAlt", "m00", "m10"]].reset_index()
    gpsMS['gpsMS'] = pd.to_datetime(gpsMS['gpsMS'], unit='ms')
    rfMS = df[df.rfMS.shift() != df.rfMS][["rfMS","rf","m22"]].reset_index(drop=True)
    rfMS['rfMS'] = pd.to_datetime(rfMS['rfMS'], unit='ms')
    
    

    gps = geodetic2ned(gpsMS.lat, gpsMS.lng, gpsMS.gpAlt)
    gps_north = gps.North.diff().fillna(0)/(gpsMS.gpsMS.diff().dt.total_seconds())
    gps_east = gps.East.diff().fillna(0)/(gpsMS.gpsMS.diff().dt.total_seconds())
    gps_down = gps.Down.diff().fillna(0)/100/(gpsMS.gpsMS.diff().dt.total_seconds())
    
    #Convert optical flow and rangefinder to OF Velocity in body frame
    of_vel_bf = pd.DataFrame(data={"OF Forward":(ofMS.frF - ofMS.brF) * ofMS.rf/100,
                                   "OF Right":(ofMS.frR - ofMS.brR) * ofMS.rf/100})
    earth_frame = body_to_earth2D(of_vel_bf["OF Forward"], of_vel_bf["OF Right"], ofMS.m00, ofMS.m10)
    
    lpf_E = -low_pass_filter(earth_frame.East, alpha=0.2)
    lpf_N = low_pass_filter(earth_frame.North, alpha=0.2)
    
    gps_east = pd.Series(gps_east)
    lpf_E = pd.Series(lpf_E)
    gps_north = pd.Series(gps_north)
    lpf_N = pd.Series(lpf_N)
    
    
    #Matching OF to the GPS update rate
    of_east = signal_match_and_cumsum(ofMS.ofMS.diff().dt.total_seconds().fillna(0).cumsum()[1:].reset_index(drop=True), change_in_signal(lpf_E),
                                      gpsMS.gpsMS.diff().dt.total_seconds().fillna(0).cumsum(), gps_east)
    of_east = pd.Series(of_east, name="OF East, LPF")
    of_north = signal_match_and_cumsum(ofMS.ofMS.diff().dt.total_seconds().fillna(0).cumsum()[1:].reset_index(drop=True), change_in_signal(lpf_N),
                                       gpsMS.gpsMS.diff().dt.total_seconds().fillna(0).cumsum(), gps_north)
    of_north = pd.Series(of_north, name="OF North, LPF")
    
    diff_north = abs(gps_north - of_north)
    diff_east = abs(gps_east - of_east)
    max_errors.append(max(diff_north.max(), diff_east.max()))
    errors_north.append(diff_north)
    errors_east.append(diff_east)

## Approach #1, Error Margin from 4 flights, Test on 1

In [2]:
#key is the mission being tested by error derived from the other 4
false_positives = {0:0,
                   1:0,
                   2:0,
                   3:0,
                   4:0}

for error in range(len(max_errors)):
    bound = max_errors[error]
    max_bound = 0
    for others in range(len(max_errors)):
        if others == error:
            continue
        max_bound = max(max_bound, max_errors[others])
    false_positives[error] = len(errors_east[error][errors_east[error] >= max_bound].index.union(errors_north[error][errors_north[error] >= max_bound]))/len(errors_east[error])

In [3]:
false_positives

{0: 0.0, 1: 0.0038461538461538464, 2: 0.0, 3: 0.0, 4: 0.0}

## Approach #2, Error Margin from 1 flight, Tested against 4

In [4]:
#first key is where the error bound comes from,
#nested keys are the mission being tested
false_positives = {0:{1:0, 2:0, 3:0, 4:0},
                   1:{0:0, 2:0, 3:0, 4:0},
                   2:{0:0, 1:0, 3:0, 4:0},
                   3:{0:0, 1:0, 2:0, 4:0},
                   4:{0:0, 1:0, 2:0, 3:0}}

for error in range(len(max_errors)):
    bound = max_errors[error]
    for others in range(len(max_errors)):
        if others == error:
            continue
        east_fp = errors_east[others][errors_east[others] >= bound].index
        north_fp = errors_north[others][errors_north[others] >= bound].index
        false_positives[error][others] = len(east_fp.union(north_fp))/len(errors_east[others])

In [5]:
import pprint
pprint.pprint(false_positives)

{0: {1: 0.011538461538461539,
     2: 0.011673151750972763,
     3: 0.05627705627705628,
     4: 0.019305019305019305},
 1: {0: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
 2: {0: 0.0, 1: 0.011538461538461539, 3: 0.0, 4: 0.003861003861003861},
 3: {0: 0.0,
     1: 0.011538461538461539,
     2: 0.0038910505836575876,
     4: 0.003861003861003861},
 4: {0: 0.0, 1: 0.0038461538461538464, 2: 0.0, 3: 0.0}}
